## Segmenting and Clustering Neighborhoods in Toronto

##### Ruud Meulendijks - Capstone Project: Week 3 assignment

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    lib

In [2]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 19.6MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from bs4 import BeautifulSoup

In [8]:
#!/usr/bin/env python3
# Anchor extraction from HTML document
###### from bs4 import BeautifulSoup
from urllib.request import urlopen

with urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as response:
    soup = BeautifulSoup(response, 'html.parser')
    for anchor in soup.find_all('a'):
        print(anchor.get('href', '/'))

/
#mw-head
#p-search
/wiki/Postal_codes_in_Canada
/wiki/Toronto
/wiki/Ontario
/wiki/Canada_Post
#cite_note-1
/wiki/Mobile_app
/wiki/Smartphones
/wiki/IPhone
/wiki/BlackBerry
#cite_note-2
/wiki/CD-ROM
/wiki/Toronto
/wiki/Postal_codes_in_Canada#Forward_sortation_areas
/w/index.php?title=List_of_postal_codes_of_Canada:_M&action=edit&section=1
/wiki/Amazon_(company)
/wiki/North_York
/wiki/Parkwoods
/wiki/North_York
/wiki/Victoria_Village
/wiki/Downtown_Toronto
/wiki/Regent_Park
/wiki/North_York
/wiki/Lawrence_Heights
/wiki/North_York
/wiki/Lawrence_Manor
/wiki/Queen%27s_Park_(Toronto)
/wiki/Downtown_Toronto
/wiki/Queen%27s_Park_(Toronto)
/wiki/Scarborough,_Toronto
/wiki/Rouge,_Toronto
/wiki/Scarborough,_Toronto
/wiki/Malvern,_Toronto
/wiki/North_York
/wiki/East_York
/wiki/Woodbine_Gardens
/wiki/East_York
/wiki/Parkview_Hill
/wiki/Downtown_Toronto
/wiki/Downtown_Toronto
/wiki/North_York
/wiki/Etobicoke
/wiki/Etobicoke
/wiki/Islington,_Toronto
/wiki/Etobicoke
/wiki/Etobicoke
/wiki/Princess_G

In [9]:
table = soup.find('table',{'class':'wikitable sortable'})
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [20]:
table1 = soup.find("table")
columns = ["Postalcode", "Borough", "Neighborhood"]
df = pd.DataFrame(columns = columns)

In [25]:
for tr_cell in table1.find_all("tr"):
    row_data=[]
    for td_cell in tr_cell.find_all("td"):
        row_data.append(td_cell.text.strip())
    if len(row_data) == 3:
        df.loc[len(df)] = row_data

df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [31]:
indexNames = df[ df["Borough"] == "Not assigned"].index
df.drop(indexNames , inplace=True)
df.loc[df["Neighborhood"] == "Not assigned", "Neighborhood"] = df["Borough"]
result = df.groupby(['Postalcode','Borough'], sort=False).agg(", ".join)
df_final = result.reset_index()
df_final.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods, Parkwoods, Pa..."
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Harb..."
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park, Quee..."
5,M9A,Downtown Toronto,"Queen's Park, Queen's Park, Queen's Park, Quee..."
6,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern..."
7,M3B,North York,"Don Mills North, Don Mills North, Don Mills No..."
8,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard..."
9,M5B,Downtown Toronto,"Ryerson, Garden District, Ryerson, Garden Dist..."


In [34]:
address = "Toronto, Ontario, Canada"

geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto, CA are {}, {}.".format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.


In [38]:
!wget -q -O "Toronto_long_lat_data.csv"  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv("Toronto_long_lat_data.csv")
df_lon_lat.columns=["Postalcode", "Latitude", "Longitude"]
df_Toronto = pd.merge(df_final, df_lon_lat[["Postalcode", "Latitude", "Longitude"]], on = "Postalcode")
df_Toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods, Parkwoods, Pa...",43.753259,-79.329656
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Harb...",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei...",43.718518,-79.464763
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park, Quee...",43.662301,-79.389494


In [42]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto["Latitude"], df_Toronto["Longitude"], df_Toronto["Borough"], df_Toronto["Neighborhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = "blue",
        fill = True,
        fill_color = "#3186cc",
        fill_opacity = 0.7,
        parse_html = False).add_to(toronto_map)  
    
toronto_map

In [45]:
di_tneilc = 'LZUCVFNJWWPWQGMC23XYS0NQI550YVVYLIMVOG4LOGWI2Z0T'
terces_tneilc = 'EZOF2SR2BMNLLJRPVPWSFD0K3AYKLABR0CTELNHC0Z5UMBMI'
noisrev = '20180605'

In [63]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            di_tneilc, 
            terces_tneilc, 
            noisrev, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

Parkwoods, Parkwoods, Parkwoods, Parkwoods, Parkwoods
Victoria Village, Victoria Village, Victoria Village, Victoria Village, Victoria Village
Harbourfront, Harbourfront, Harbourfront, Harbourfront, Harbourfront
Lawrence Heights, Lawrence Manor, Lawrence Heights, Lawrence Manor, Lawrence Heights, Lawrence Manor, Lawrence Heights, Lawrence Manor, Lawrence Heights, Lawrence Manor
Queen's Park, Queen's Park, Queen's Park, Queen's Park, Queen's Park
Queen's Park, Queen's Park, Queen's Park, Queen's Park, Queen's Park
Rouge, Malvern, Rouge, Malvern, Rouge, Malvern, Rouge, Malvern, Rouge, Malvern
Don Mills North, Don Mills North, Don Mills North, Don Mills North, Don Mills North
Woodbine Gardens, Parkview Hill, Woodbine Gardens, Parkview Hill, Woodbine Gardens, Parkview Hill, Woodbine Gardens, Parkview Hill, Woodbine Gardens, Parkview Hill
Ryerson, Garden District, Ryerson, Garden District, Ryerson, Garden District, Ryerson, Garden District, Ryerson, Garden District
Glencairn, Glencairn, Gle

In [65]:
print(toronto_venues.shape)
toronto_venues.head()

(2213, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkwoods, Parkwoods, Parkwoods, Parkwoods, Pa...",43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,"Parkwoods, Parkwoods, Parkwoods, Parkwoods, Pa...",43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [66]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [67]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [68]:
toronto_onehot.shape

(2213, 273)

In [69]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond, Adelaide, King, Rich...",0.000000,0.000000,0.000000,0

In [70]:
toronto_grouped.shape

(100, 273)

In [71]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond, Adelaide, King, Richmond, Adelaide, King, Richmond, Adelaide, King, Richmond, Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4       Restaurant  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground  0.33
1                Park  0.33
2    Asian Restaurant  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Agincourt, Agincourt, Agincourt, Agincourt, Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Sandwich Place  0.25
4      

In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond, Adelaide, King, Rich...",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bakery,Restaurant,Salad Place,Burger Joint,Sushi Restaurant,Asian Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"Agincourt, Agincourt, Agincourt, Agincourt, Ag...",Latin American Restaurant,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Sandwich Place,Liquor Store,Pizza Place,Pharmacy,Colombian Restaurant,Dessert Shop
4,"Alderwood, Long Branch, Alderwood, Long Branch...",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,Gay Bar,General Entertainment,Doner Restaurant


In [76]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [117]:
toronto_merged = df_Toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged=toronto_merged.dropna()
convert_dict = {'Cluster Labels': int} 
toronto_merged = toronto_merged.astype(convert_dict) 
toronto_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods, Parkwoods, Pa...",43.753259,-79.329656,4,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M4A,North York,"Victoria Village, Victoria Village, Victoria V...",43.725882,-79.315572,0,Hockey Arena,Coffee Shop,Intersection,Portuguese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Harb...",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Hotel
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei...",43.718518,-79.464763,0,Clothing Store,Accessories Store,Women's Store,Coffee Shop,Furniture / Home Store,Arts & Crafts Store,Carpet Store,Miscellaneous Shop,Vietnamese Restaurant,Boutique
4,M7A,Queen's Park,"Queen's Park, Queen's Park, Queen's Park, Quee...",43.662301,-79.389494,0,Coffee Shop,Gym,Park,Diner,Wings Joint,Sandwich Place,Restaurant,Portuguese Restaurant,Persian Restaurant,Nightclub
5,M9A,Downtown Toronto,"Queen's Park, Queen's Park, Queen's Park, Quee...",43.667856,-79.532242,0,Coffee Shop,Gym,Park,Diner,Wings Joint,Sandwich Place,Restaurant,Portuguese Restaurant,Persian Restaurant,Nightclub
6,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern...",43.806686,-79.194353,3,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
7,M3B,North York,"Don Mills North, Don Mills North, Don Mills No...",43.745906,-79.352188,0,Café,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Gym / Fitness Center,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
8,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard...",43.706397,-79.309937,0,Pizza Place,Fast Food Restaurant,Bank,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Athletics & Sports,Intersection,Women's Store
9,M5B,Downtown Toronto,"Ryerson, Garden District, Ryerson, Garden Dist...",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Lingerie Store,Bakery,Restaurant,Pizza Place


In [113]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
      
map_clusters

#### First cluster

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Hockey Arena,Coffee Shop,Intersection,Portuguese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Theater,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Hotel
3,North York,0.0,Clothing Store,Accessories Store,Women's Store,Coffee Shop,Furniture / Home Store,Arts & Crafts Store,Carpet Store,Miscellaneous Shop,Vietnamese Restaurant,Boutique
4,Queen's Park,0.0,Coffee Shop,Gym,Park,Diner,Wings Joint,Sandwich Place,Restaurant,Portuguese Restaurant,Persian Restaurant,Nightclub
5,Downtown Toronto,0.0,Coffee Shop,Gym,Park,Diner,Wings Joint,Sandwich Place,Restaurant,Portuguese Restaurant,Persian Restaurant,Nightclub
7,North York,0.0,Café,Japanese Restaurant,Caribbean Restaurant,Basketball Court,Gym / Fitness Center,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
8,East York,0.0,Pizza Place,Fast Food Restaurant,Bank,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Athletics & Sports,Intersection,Women's Store
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Lingerie Store,Bakery,Restaurant,Pizza Place
10,North York,0.0,Japanese Restaurant,Pizza Place,Bakery,Pub,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
12,Scarborough,0.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market


#### Second cluster

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,1.0,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


#### Third cluster

In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Jewelry Store,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


#### Fourth cluster

In [115]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant


#### Fifth cluster

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
21,York,4,Park,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
35,East York,4,Convenience Store,Coffee Shop,Park,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,North York,4,Park,Airport,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
64,York,4,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
66,North York,4,Convenience Store,Park,Bank,Bar,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
85,Scarborough,4,Park,Asian Restaurant,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
91,Downtown Toronto,4,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
98,Etobicoke,4,Park,Smoke Shop,River,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Dance Studio,Dog Run
